# Tutorial for HPS single DDX4 

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb

# set simulation platform
platform_name = 'CPU'

We simulate a DDX4 protein with HPS model. We begin with building a straight CA atom chain with DDX4 sequence. We use `HPSParser` to parse the CA model.

In [2]:
sequence = 'MGDEDWEAEINPHMSSYVPIFEKDRYSGENGDNFNRTPASSSEMDDGPSRRDHFMKSGFASGRNFGNRDAGECNKRDNTSTMGGFGVGKSFGNRGFSNSRFEDGDSSGFWRESSNDCEDNPTRNRGFSKRGGYRDGNNSEASGPYRRGGRGSFRGCRGGFGLGSPNNDLDPDECMQRTGGLFGSRRPVLSGTGNGDTSQSRSGSGSERGGYKGLNEEVITGSGKNSWKSEAEGGES'
ca_pdb = 'init_DDX4_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)
protein = HPSModel()
protein.append_mol(protein_parser)

Parse molecule with default settings.


Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [3]:
top = app.PDBFile(ca_pdb).getTopology()
init_coord = app.PDBFile(ca_pdb).getPositions()
protein.create_system(top, box_a=100, box_b=100, box_c=100)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 300*unit.kelvin
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
platform_name = 'CPU'
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_single_DDX4.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)

Add protein bonds.
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,381.5090961592769,791.5938839493183,1173.1029801085951,270.09038485107203,0
200,2.0000000000000013,447.22226752854186,914.0567649033711,1361.279032431913,311.87449576642217,185
300,2.99999999999998,545.5821257116296,900.1396844237371,1445.7218101353667,307.12601337038035,186
400,3.9999999999999587,498.03454525917573,964.0987293811315,1462.1332746403073,328.94872248615184,186
500,4.999999999999938,519.3133253109844,962.5798690669659,1481.8931943779503,328.4304901259656,188


In [4]:
# visualize trajectory
traj = mdtraj.load_dcd('output_single_DDX4.dcd', top=ca_pdb)
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=4)